In [5]:
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score, classification_report, roc_auc_score,matthews_corrcoef, precision_recall_curve
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.combine import SMOTEENN
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [20]:
#scikit-learn.__version__
import xgboost as x
x.__version__


#pandas oudere versie
#sklearn iets oudere versie
#imbalanced-learn oudere versie
#xgboost verouderde versie
#

'1.0.2'

## Importing datasets

In [8]:
import os
os.chdir("C:\\Users\\tsmit\\Downloads\\KPMG thesis\\Data")

dfenron=pd.read_pickle("ENRON set.zip")
dfenron['CONTENT+SUBJECT']=dfenron['CONTENT']+dfenron['SUBJECT']
dfenron=dfenron[dfenron['CONTENT']!=' ']

In [7]:
import os
os.chdir("Z:\\Scriptie TSmit\\pickles")
os.getcwd()

dfwhiskey=pd.read_pickle('Whiskey.zip')
dfwhiskey['CONTENT+SUBJECT']=[' '.join(dfwhiskey.loc[index]['TOKENS_BODY_SUBJ']) for index in dfwhiskey.index]
dfwhiskey=dfwhiskey[dfwhiskey['CONTENT']!=' ']
dfwhiskey=dfwhiskey[dfwhiskey['CONTENT']!='']

df2020=pd.read_pickle('2020.zip')
df2020['CONTENT+SUBJECT']=[' '.join(df2020.loc[index]['TOKENS_BODY_SUBJ']) for index in df2020.index]
df2020=df2020[df2020['CONTENT']!=' ']
df2020=df2020[df2020['CONTENT']!='']

# Training loop with settings 

In [174]:
def check_preprocess_grid(dataset_name):
    if dataset_name == 'enron':
        smoteenn_grid=[False]
        undersampling_grid=[True]
    else:
        smoteenn_grid=[True]
        undersampling_grid=[True,False]
    return smoteenn_grid, undersampling_grid


#true smoteenn true
# undersampling beide
# class weigts beide
#Sampleweights true

def create_score_table(dataset_name):
    enron_scores=pd.DataFrame(columns=['Text representation','Word/char','Weighting','Algorithm','Smoteenn','Undersampling','Classweights','Sampleweights'])
    a=0
    preprocess_grid_smoteenn,preprocess_grid_undersampling=check_preprocess_grid(dataset_name) 
    for representation in ['UNIGRAM','BIGRAM','TRIGRAM']:
        for wordchar in ['WORD','CHAR']:
            for weighting in ['INFO-GAIN']:
                for model in ['svm','rf','xgb','cnb']:
                    for smoteennyes in preprocess_grid_smoteenn:
                        for underyes in preprocess_grid_undersampling:  # always !! undersampling for enron 
                            for classweightyes in [True,False]:
                                for sampleweightyes in [True]:
                                    enron_scores.loc[a]=[representation,wordchar,weighting,model,smoteennyes,underyes,classweightyes,sampleweightyes]
                                    a+=1
    return enron_scores


#in case of enron training #fill in right datanames en sets!

#df2020_training_results=do_training('2020','tokens')

trainingwhiskey=create_score_table('2020')
#kijken hoe training is op GEEN lege emails. 

In [ ]:
def extract_ngram(text_representation):
    if text_representation=='UNIGRAM':
        ngram_range=(1,1)
    elif text_representation=='BIGRAM':
        ngram_range=(1,2)
    elif text_representation=='TRIGRAM':
        ngram_range=(1,3)
    return ngram_range

In [125]:
def extract_ngram(text_representation):
    if text_representation=='UNIGRAM':
        ngram_range=(1,1)
    elif text_representation=='BIGRAM':
        ngram_range=(1,2)
    elif text_representation=='TRIGRAM':
        ngram_range=(1,3)
    return ngram_range

def perform_tf_idf(X_train,X_test,ngram_range,analyzer,use_idf):
    min_df = 10
    max_df = 1.
    max_features = 300
    tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                            analyzer=analyzer,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True,
                           use_idf=use_idf)
    features_train = tfidf.fit_transform(X_train).toarray()
    features_test = tfidf.transform(X_test).toarray()
    return features_train,features_test

def perform_countvectorizer(X_train,X_test,ngram_range,analyzer):
    min_df = 10
    max_df = 1.
    max_features = 300
    countvect = CountVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,analyzer=analyzer,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features)
    features_train = countvect.fit_transform(X_train).toarray()
    features_test = countvect.transform(X_test).toarray()
    return features_train,features_test    

def perform_info_gain_df(X_train,X_test,labels,ngram_range,analyzer):
    from info_gain import info_gain
    X_train=X_train.astype(str)
    X_test=X_test.astype(str)
    min_df = 10
    max_df = 1.
    max_features = 300
    countvect = CountVectorizer(encoding='utf-8',ngram_range=ngram_range,analyzer=analyzer,stop_words=None,lowercase=False,max_df=max_df,min_df=min_df,max_features=max_features,binary=True)
    features_train = countvect.fit_transform(X_train).toarray()
    features_test = countvect.transform(X_test).toarray()
    
    dftrain = pd.DataFrame(features_train, columns=countvect.get_feature_names())
    dftest = pd.DataFrame(features_test, columns=countvect.get_feature_names())
    
    dftrain['RESPONSIVE']=labels
    
    
    ig_dict={}
    for column in dftrain.columns:
        ig  = info_gain.info_gain(dftrain['RESPONSIVE'], dftrain[column])
        ig_dict[column]=ig
    
    for column in dftrain.columns:
        info_gain=ig_dict[column]
        dftrain[column]=np.where(dftrain[column] == 0, 0, info_gain)
    for column in dftest.columns:
        info_gain=ig_dict[column]
        dftest[column]=np.where(dftest[column] == 0, 0, info_gain)
    return dftrain.loc[:, dftrain.columns != 'RESPONSIVE'],dftest

    
def train_classifier(data,model,iterations,cv,smoteennyes,underyes,classweightyes,sampleweightyes,text_representation,wordchar,feature_weighting):
    X_train, X_test, y_train, y_test = train_test_split(data['TOKENS'],   #ALLEEN VOOR ENRON
                                                    data['RESPONSIVE'], 
                                                    test_size=0.15)
    X_train=X_train.astype(str)
    X_test=X_test.astype(str)    #weet niet zeker of dit moet
    
    ngram_range = extract_ngram(text_representation)
    
    print(feature_weighting)
    if feature_weighting =='TF-IDF':
        features_train,features_test=perform_tf_idf(X_train,X_test,ngram_range,wordchar.lower(),True)
    elif feature_weighting =='TF':
        features_train,features_test=perform_tf_idf(X_train,X_test,ngram_range,wordchar.lower(),False)
    elif feature_weighting=='BINARY':
        features_train,features_test=perform_countvectorizer(X_train,X_test,ngram_range,wordchar.lower())
    elif feature_weighting=='INFO-GAIN':
        features_train,features_test=perform_info_gain_df(X_train,X_test,y_train,ngram_range,wordchar.lower())
    
    labels_train = y_train
    labels_test = y_test
    
    print('na vectorizer')
    if smoteennyes:
        print('start smoteenn')
        sme=SMOTEENN()
        features_train, labels_train = sme.fit_resample(features_train, labels_train)
        print('SMOTEENN: True')
        
    if underyes:
        under=RandomUnderSampler(sampling_strategy=1)
        features_train, labels_train =under.fit_resample(features_train,labels_train)
        print('Undersamping: True')
        
    if classweightyes:
        class_weight='balanced'
    else:
        class_weight=None
    print('class_weight: ', class_weight)
    
    if model=='svm':
        print('Support vector machines')
        estimator= SVC(random_state=8)
        random_grid=svm_grid
    elif model=='rf':
        print('Random forest')
        estimator= RandomForestClassifier(random_state=8)
        random_grid=rf_grid
    elif model=='cnb':
        print('Complement Naive Bayes')
        estimator= ComplementNB()
        random_grid=cnb_grid
    elif model=='xgb':
        print('XGBoost')
        estimator=XGBClassifier(random_state=8)
        ratio_w=len(data[data['RESPONSIVE']==0])/len(data[data['RESPONSIVE']==1])
        random_grid=xgb_grid
    
   
    random_search = RandomizedSearchCV(estimator=estimator,
                                   param_distributions=random_grid,
                                   n_iter=iterations,
                                   scoring=ftwo_scorer,
                                   cv=cv, 
                                   verbose=1, 
                                   refit=True)
    if sampleweightyes:
        print('Sample weight: True')
        sample_weight = compute_sample_weight(class_weight='balanced', y=labels_train)
        print('start random search')
        random_search.fit(features_train, labels_train,sample_weight=sample_weight)
        print('end random search')
    else:
        print('start random search')
        random_search.fit(features_train, labels_train)
        print('end random search')
    print('--------------------------------------------------------------')
    print('Training score:')
    print("The best hyperparameters from Random Search are:")
    print(random_search.best_params_)
    print("")
    print("The mean f2 of a model with these hyperparameters is:")
    print(random_search.best_score_)
    print("")
    print('--------------------------------------------------------------')
    print('Test score:')
    y_pred=random_search.predict(features_test)
    y_probas=random_search.predict_proba(features_test)
    print("")
    print("Confusion matrix")
    print(confusion_matrix(labels_test,y_pred))
    print(classification_report(labels_test, y_pred))
    print('')
    print("F1: ",f1_score(labels_test,y_pred))
    print("ROC-AUC: ",roc_auc_score(labels_test,y_pred))
    print('Matthews corr coef: ',matthews_corrcoef(labels_test,y_pred))
    f2_score_before_thresholding=fbeta_score(labels_test,y_pred,beta=2)
    p, r, thresholds = precision_recall_curve(y_test, y_probas[:,1])

    f2_list=[]
    for i in range(0,len(p)):
        f2_list.append((5*p[i]*r[i])/((4*p[i])+r[i]))

    t=thresholds[f2_list.index(max(f2_list))]
    new_labels=adjusted_classes(y_probas[:,1],t)
    f2_score=fbeta_score(y_test,new_labels,beta=2)
    
    return f1_score(labels_test,y_pred),f2_score,f2_score_before_thresholding,random_search.best_params_

class_weight='balanced'
ratio_w=1

svm_grid = {'C': [.01,.1,.4,.6,1,1.5],
              'kernel': [ 'rbf', 'poly','linear'],
              'gamma':  [.01, .1, 1],
              'degree': [1, 2, 3, 4, 5],
              'probability': [True],
              'class_weight':[class_weight]
             }

xgb_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'scale_pos_weight':[ratio_w] 
}

rf_grid={'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False],
                'class_weight':['balanced']}

cnb_grid={'alpha':[0,0.3,0.6,0.8,1], 
          'fit_prior':[False,True],
          'norm':[False,True]}


def adjusted_classes(y_scores, t):
    """
    This function adjusts class predictions based on the prediction threshold (t).
    """
    return [1 if y >= t else 0 for y in y_scores]


ftwo_scorer = make_scorer(fbeta_score, beta=2)

## Testing results

In [126]:
#enter parameter combination in de estimator!


def do_testing(data,model,best_params,smoteennyes,underyes,classweightyes,sampleweightyes,text_representation,wordchar,feature_weighting):
    X_train, X_test, y_train, y_test = train_test_split(data['TOKENS'], 
                                                    data['RESPONSIVE'], 
                                                    test_size=0.15,stratify=data['RESPONSIVE'])
                                                    
    X_train_1, X_val, y_train_1, y_val=train_test_split(X_train,  
                                                    y_train, 
                                                    test_size=0.15,stratify=y_train)               
    print('verhouding in train qua labels: ', y_train.value_counts()[0]/y_train.value_counts()[1])
    print('verhouding in test qua labels: ', y_test.value_counts()[0]/y_test.value_counts()[1])
    
    def fit_and_val_model(model,best_params,X_train_1,y_train_1,X_val,y_val,smoteennyes,underyes,classweightyes,sampleweightyes,text_representation,wordchar,feature_weighting):
        X_train_1=X_train_1.astype(str)
        X_val=X_val.astype(str)    #weet niet zeker of dit moet
    
        ngram_range = extract_ngram(text_representation)
        if feature_weighting =='TF-IDF':
            X_train_1,X_val=perform_tf_idf(X_train_1,X_val,ngram_range,wordchar.lower(),True)
        elif feature_weighting =='TF':
            X_train_1,X_val=perform_tf_idf(X_train_1,X_val,ngram_range,wordchar.lower(),False)
        elif feature_weighting=='BINARY':
            X_train_1,X_val=perform_countvectorizer(X_train_1,X_val,ngram_range,wordchar.lower())
        elif feature_weighting=='INFO-GAIN':
            X_train_1,X_val=perform_info_gain_df(X_train_1,X_val,y_train_1,ngram_range,wordchar.lower())
            
        y_train_1 = y_train_1
        y_val = y_val
        
        if smoteennyes:
            print('start smoteenn')
            sme=SMOTEENN()
            X_train_1, y_train_1 = sme.fit_resample(X_train_1, y_train_1)
            print('SMOTEENN: True')

        if underyes:
            under=RandomUnderSampler(sampling_strategy=1)
            X_train_1,y_train_1 =under.fit_resample(X_train_1,y_train_1)
            print('Undersamping: True')

        if classweightyes:
            class_weight='balanced'
        else:
            class_weight=None
        print('class_weight: ', class_weight)
        
        
        if model=='svm':
            estimator= SVC(C=best_params['C'],kernel=best_params['kernel'],gamma=best_params['gamma'],degree=best_params['degree'],probability=best_params['probability'],class_weight=class_weight)
        elif model=='rf':
            estimator= RandomForestClassifier(n_estimators=best_params['n_estimators'],max_depth=best_params['max_depth'],min_samples_split=best_params['min_samples_split'],min_samples_leaf=best_params['min_samples_leaf'],bootstrap=best_params['bootstrap'],class_weight='balanced')
        elif model=='cnb':
            estimator= ComplementNB(alpha=best_params['alpha'],fit_prior=best_params['fit_prior'],norm=best_params['norm'])
        elif model=='xgb':
            ratio_w=y_train_1.value_counts()[0]/y_train_1.value_counts()[1]
            estimator=XGBClassifier(min_child_weight=best_params['min_child_weight'],gamma=best_params['gamma'],subsample=best_params['subsample'],colsample_bytree=best_params['colsample_bytree'],max_depth=best_params['max_depth'], scale_pos_weight=ratio_w)
         
    
        if sampleweightyes:
            sample_weight = compute_sample_weight(class_weight='balanced', y=y_train_1)
            print('start random search')
            estimator.fit(X_train_1, y_train_1,sample_weight=sample_weight)
            print('end random search')
        else:
            estimator.fit(X_train_1,y_train_1)

        y_pred=estimator.predict(X_val)
        y_probas=estimator.predict_proba(X_val)
        f2_score_before_thresholding=fbeta_score(y_val,y_pred,beta=2)
        p, r, thresholds = precision_recall_curve(y_val, y_probas[:,1])
        f2_list=[]
        for i in range(0,len(p)):
            f2_list.append((5*p[i]*r[i])/((4*p[i])+r[i]))
        t=thresholds[f2_list.index(max(f2_list))]
        new_labels=adjusted_classes(y_probas[:,1],t)
        f2_score=fbeta_score(y_val,new_labels,beta=2)

        print('F2 before thresholding: ',f2_score_before_thresholding)
        print('F2 after thresholding: ', f2_score)
        print(t)
        print('----------------------------------')
        return t

    def test_model(model,best_params,X_train,y_train,X_test,y_test,t,smoteennyes,underyes,classweightyes,sampleweightyes,text_representation,wordchar,feature_weighting):
        X_train=X_train.astype(str)
        X_test=X_test.astype(str)    #weet niet zeker of dit moet

        ngram_range = extract_ngram(text_representation)
        
        if feature_weighting =='TF-IDF':
            X_train,X_test=perform_tf_idf(X_train,X_test,ngram_range,wordchar.lower(),True)
        elif feature_weighting =='TF':
            X_train,X_test=perform_tf_idf(X_train,X_test,ngram_range,wordchar.lower(),False)
        elif feature_weighting=='BINARY':
            X_train,X_test=perform_countvectorizer(X_train,X_test,ngram_range,wordchar.lower())
        elif feature_weighting=='INFO-GAIN':
            X_train,X_test=perform_info_gain_df(X_train,X_test,y_train,ngram_range,wordchar.lower())   
        
        y_train = y_train
        y_test = y_test
        
        if smoteennyes:
            print('start smoteenn')
            sme=SMOTEENN()
            X_train, y_train = sme.fit_resample(X_train, y_train)
            print('SMOTEENN: True')

        if underyes:
            under=RandomUnderSampler(sampling_strategy=1)
            X_train,y_train =under.fit_resample(X_train,y_train)
            print('Undersamping: True')

        if classweightyes:
            class_weight='balanced'
        else:
            class_weight=None
        print('class_weight: ', class_weight)
        
        if model=='svm':
            estimator= SVC(C=best_params['C'],kernel=best_params['kernel'],gamma=best_params['gamma'],degree=best_params['degree'],probability=best_params['probability'],class_weight=class_weight)
        elif model=='rf':
            estimator= RandomForestClassifier(n_estimators=best_params['n_estimators'],max_depth=best_params['max_depth'],min_samples_split=best_params['min_samples_split'],min_samples_leaf=best_params['min_samples_leaf'],bootstrap=best_params['bootstrap'],class_weight='balanced')
        elif model=='cnb':
            estimator= ComplementNB(alpha=best_params['alpha'],fit_prior=best_params['fit_prior'],norm=best_params['norm'])
        elif model=='xgb':
            ratio_w=y_train.value_counts()[0]/y_train.value_counts()[1]
            estimator=XGBClassifier(min_child_weight=best_params['min_child_weight'],gamma=best_params['gamma'],subsample=best_params['subsample'],colsample_bytree=best_params['colsample_bytree'],max_depth=best_params['max_depth'], scale_pos_weight=ratio_w)
         
         
        if sampleweightyes:
            sample_weight = compute_sample_weight(class_weight='balanced', y=y_train)
            estimator.fit(X_train, y_train,sample_weight=sample_weight)
        else:
            estimator.fit(X_train,y_train)
            
        y_pred=estimator.predict(X_test)
        y_probas=estimator.predict_proba(X_test)
        f2_score_without_thresholding=fbeta_score(y_test,y_pred,beta=2)

        new_labels=adjusted_classes(y_probas[:,1],t)
        f2_score=fbeta_score(y_test,new_labels,beta=2)

        print('test f2 without thresholding: ',f2_score_without_thresholding)
        print('test f2 after thresholding: ',f2_score)
        return f2_score, f2_score_without_thresholding

    t=fit_and_val_model(model,best_params,X_train_1,y_train_1,X_val,y_val,smoteennyes,underyes,classweightyes,sampleweightyes,text_representation,wordchar,feature_weighting)
    return test_model(model,best_params,X_train,y_train,X_test,y_test,t,smoteennyes,underyes,classweightyes,sampleweightyes,text_representation,wordchar,feature_weighting)


#test configuration on multiple test sets to check reliability

def testing_statistics(data,model,best_params,iterations,smoteennyes,underyes,classweightyes,sampleweightyes,text_representation,wordchar,feature_weighting):  #oneven aantal!!!
    f2_thresh_scores=[]
    f2_no_thresh_scores=[]
    
    for iter in range(0,iterations):
        f2_thresh,f2_no_thresh=do_testing(data,model,best_params,smoteennyes,underyes,classweightyes,sampleweightyes,text_representation,wordchar,feature_weighting)
        f2_thresh_scores.append(f2_thresh)
        f2_no_thresh_scores.append(f2_no_thresh)
    return f2_thresh_scores, f2_no_thresh_scores

In [ ]:
# na train_classifier ook nog alles aanpassen van testing class

In [ ]:
ext representation	Word/char	Weighting	Settings	Smoteenn	Undersampling	Classweights	Sampleweights

In [208]:
def testing_results(training_results,data,dataset_name,tijdelijke_index):
    #training_results['F2 test with thresholding (mean)']=0
    #training_results['F2 test with thresholding (median)']=0
    #training_results['F2 test without thresholding (mean)']=0
    #training_results['F2 test without thresholding (median)']=0
    
    training_results['best params']=0
    print(len(training_results))
    for index in tijdelijke_index[45:50]:

        feature_weighting=training_results.loc[index]['Weighting']
        word_char=training_results.loc[index]['Word/char']
        text_representation=training_results.loc[index]['Text representation']
        smoteennyes=training_results.loc[index]['Smoteenn']
        underyes=training_results.loc[index]['Undersampling']#==' True ')#change tooo bolean
        classweightyes=training_results.loc[index]['Classweights']#==' True ')
        sampleweightyes=training_results.loc[index]['Sampleweights']#==' True ')
        algorithm=training_results.loc[index]['Algorithm']
        from sklearn.metrics import confusion_matrix, f1_score, classification_report, roc_auc_score,matthews_corrcoef, precision_recall_curve
        f1_score,f2_score,f2_before,best_params=train_classifier(data,algorithm,30,4,smoteennyes,underyes,classweightyes,sampleweightyes,text_representation,word_char,feature_weighting)
        f2_thresh_scores,f2_no_thresh_scores=testing_statistics(data,algorithm, best_params,11,smoteennyes,underyes,classweightyes,sampleweightyes,text_representation,word_char,feature_weighting)
        training_results.loc[index,'F2 test with thresholding (mean)']=np.mean(f2_thresh_scores)
        training_results.loc[index,'F2 test with thresholding (median)']=np.median(f2_thresh_scores)
        training_results.loc[index,'F2 test without thresholding (mean)']=np.mean(f2_no_thresh_scores)
        training_results.loc[index,'F2 test without thresholding (median)']=np.median(f2_no_thresh_scores)
        training_results.loc[index,'best params']=[best_params]
        os.chdir("C:\\Users\\tsmit\\Downloads\\KPMG thesis\\Results\\Results per feature weighting")
        training_results.to_excel('ALMIGHTY 288 laatste 4 resterende rows INFO-GAIN - test results {}.xlsx'.format(dataset_name))
        print('--------------------------------------------------------------- {} DONE'.format(index))
        #except:
            #print('{} ging niet goed-------------------------------------------------'.format(index))
    os.chdir("C:\\Users\\tsmit\\Downloads\\KPMG thesis\\Results\\Results per feature weighting")
    training_results.to_excel('ALMIGHTY 288 laatste 4 resterende rows INFO-GAIN - test results {}.xlsx'.format(dataset_name)) 
    return training_results

## call testing function for testing results

In [209]:
dfwhiskey_testing_results=testing_results(trainingwhiskey,dfwhiskey,'whiskey',tijdelijke_index)

96
INFO-GAIN
na vectorizer
start smoteenn
SMOTEENN: True
Undersamping: True
class_weight:  None
XGBoost
Sample weight: True
start random search
Fitting 4 folds for each of 30 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: feature_names must be string, and may not contain [, ] or <

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    4.4s finished


ValueError: feature_names must be string, and may not contain [, ] or <

In [176]:
#df2020_testing_results.groupby(['Text representation','Word/char']).max()

#a=df2020_testing_results.loc[df2020_testing_results.groupby(['Text representation','Word/char'])["F2 test with thresholding (mean)"].idxmax()]
#a.groupby(['Text representation','Word/char']).max()

#os.chdir("C:\\Users\\tsmit\\Downloads\\KPMG thesis\\Results\\Results per feature weighting")
#best2020=pd.read_excel('ALMIGHTY 288 rows - test results 2020.xlsx')

In [187]:
tijdelijke_index=dfwhiskey_testing_results[dfwhiskey_testing_results['best params']==0].index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 56, 57, 58, 59, 88, 89, 90, 91],
           dtype='int64')

In [205]:
tijdelijke_index[44:50]

Int64Index([88, 89, 90, 91], dtype='int64')

In [175]:
trainingwhiskey

,Text representation,Word/char,Weighting,Algorithm,Smoteenn,Undersampling,Classweights,Sampleweights
0,UNIGRAM,WORD,INFO-GAIN,svm,True,True,True,True
1,UNIGRAM,WORD,INFO-GAIN,svm,True,True,False,True
2,UNIGRAM,WORD,INFO-GAIN,svm,True,False,True,True
3,UNIGRAM,WORD,INFO-GAIN,svm,True,False,False,True
4,UNIGRAM,WORD,INFO-GAIN,rf,True,True,True,True
...,...,...,...,...,...,...,...,...
91,TRIGRAM,CHAR,INFO-GAIN,xgb,True,False,False,True
92,TRIGRAM,CHAR,INFO-GAIN,cnb,True,True,True,True
93,TRIGRAM,CHAR,INFO-GAIN,cnb,True,True,False,True
94,TRIGRAM,CHAR,INFO-GAIN,cnb,True,False,True,True
